# 💳 Task 4 – Behavioral Risk Feature Engineering for BNPL Credit Scoring  
📘 Version: 2025-07-03  

This notebook designs and implements a **feature engineering pipeline** for **Bati Bank’s BNPL Credit Scoring Initiative**, transforming raw transactional data into a **model-ready feature set** for risk prediction.  

It builds upon the outputs from **Task 3 – Proxy Label Engineering** by creating structured, explainable features that capture key dimensions of customer behavior, spending patterns, and engagement risk. These features will serve as the inputs to the predictive models in **Task 5 – Model Training & Evaluation**.  

---  

**Challenge:** B5W5 – Credit Scoring & Risk Modeling  
**Company:** Bati Bank (in partnership with an eCommerce platform)  
**Author:** Nabil Mohamed  
**Branch:** `task-4-feature-engineering`  
**Date:** July 2025  

---  

## 📌 Notebook Outline  

| Section | Description |
|---------|-------------|
| 1️⃣ Imports & Project Setup | Load libraries, define file paths, ensure reproducibility |
| 2️⃣ Load Enriched Dataset | Load transaction dataset with RFM metrics and proxy labels |
| 3️⃣ Aggregate Feature Engineering | Generate customer-level transaction aggregates (count, value, variance) |
| 4️⃣ Temporal Feature Engineering | Extract time-based behavioral patterns (recency, frequency, active hours) |
| 5️⃣ Categorical Encoding | Apply encoding to key categorical predictors |
| 6️⃣ Missing Value Handling & Scaling | Apply robust imputation and feature scaling |
| 7️⃣ Feature Set Assembly | Merge engineered features with proxy labels for modeling |
| 8️⃣ Save Model-Ready Dataset | Export final dataset for Task 5 modeling |
| 9️⃣ Next Steps Preview | Outline transition to model development and explainability |

---  

## ✏️ Key Design Principles:  

- Focus on **behavioral risk signals**—turning transactions into defensible borrower profiles.  
- Prioritize **model explainability** by including interpretable features alongside any complex transformations (e.g., Weight of Evidence).  
- Ensure **pipeline reproducibility** using **sklearn’s Pipeline** and **modular design** for seamless integration with Task 5.  
- All outputs must be **ready for immediate modeling and SHAP explainability**, minimizing technical debt in downstream tasks.  


In [6]:
# ------------------------------------------------------------------------------
# 🛠 Ensure Notebook Runs from Project Root (for src/ imports to work)
# ------------------------------------------------------------------------------

import os
import sys

# If running from /notebooks/, move up to project root
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
    print("📂 Changed working directory to project root")

# Add project root to sys.path so `src/` modules can be imported
project_root = os.getcwd()
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"✅ Added to sys.path: {project_root}")

# Optional: verify file presence to confirm we're in the right place
expected_path = "data/raw"
print(
    "📁 Output path ready"
    if os.path.exists(expected_path)
    else f"⚠️ Output path not found: {expected_path}"
)

📁 Output path ready


## 📦 Imports & Environment Setup

This cell loads the core libraries required for **feature engineering, data transformation, and preprocessing** for **Bati Bank’s BNPL Credit Risk Modeling**.

Imports are grouped by function:

- **Data handling:** `pandas`, `numpy`
- **Visualization:** `matplotlib`, `seaborn`
- **Date/time analysis:** `datetime`, `pandas.to_datetime`
- **Preprocessing & encoding:** `scikit-learn` for aggregation, scaling, and encoding
- **System & utilities:** `os`, `warnings`, `pathlib`

The environment is configured for **reproducibility, clean visuals, and clear output formatting** for downstream model training and explainability.


In [7]:
# ---------------------------
# 📦 Imports & Environment Setup
# ---------------------------

# Data handling
import pandas as pd  # For data manipulation and analysis
import numpy as np  # For numerical operations

# Visualization
import matplotlib.pyplot as plt  # For plotting static visualizations
import seaborn as sns  # For enhanced statistical visualizations

# Date/time analysis
from datetime import datetime  # For working with date and time objects
from pandas.api.types import CategoricalDtype  # For defining ordered categories

# Preprocessing & encoding
from sklearn.preprocessing import StandardScaler  # For standardizing numerical features
from sklearn.preprocessing import (
    OneHotEncoder,
)  # For one-hot encoding of categorical features
from sklearn.preprocessing import (
    LabelEncoder,
)  # For label encoding of categorical features
from sklearn.pipeline import (
    Pipeline,
)  # For creating reproducible transformation pipelines
from sklearn.impute import SimpleImputer  # For missing value imputation

# System & utilities
import os  # For file path operations
import warnings  # For suppressing warnings
from pathlib import Path  # For handling file system paths

# Configure display settings for pandas
pd.set_option("display.max_columns", None)  # Show all columns in DataFrame outputs
pd.set_option(
    "display.float_format", "{:,.2f}".format
)  # Format floats to 2 decimal places

# Ignore non-critical warnings to keep output clean
warnings.filterwarnings("ignore")

# Set seaborn style for consistent visuals
sns.set(
    style="whitegrid", context="notebook"
)  # Apply white grid style for better readability

## 📥 Load & Preview Enriched Transaction Dataset (Task 4 Feature Engineering)

This step loads the **enriched transaction dataset** (`enriched_transactions_with_proxy_labels.csv`) prepared in **Task 3 – Proxy Label Engineering**.  
This dataset includes behavioral features such as **Recency, Frequency, Monetary (RFM) metrics**, **fraud tags**, and the generated **proxy risk labels** (`risk_category`, `is_high_risk`).

Key actions in this step:
- Load the **proxy-labeled** dataset from `data/processed/enriched_transactions_with_proxy_labels.csv` using the `CreditDataLoader` utility.
- Verify data structure: row count, columns, data types, sample preview.
- Provide clear diagnostics and robust error handling.
- Ensure compatibility with downstream **feature engineering and model training** in Task 5.

---


In [8]:
# ------------------------------------------------------------------------------
# 📦 Load Enriched Credit Transaction Data (Task 4 Loader)
# ------------------------------------------------------------------------------

# Import the custom CreditDataLoader from src
from src.data_loader import CreditDataLoader  # Defensive, reusable data loader class

# Define the path to the enriched transaction dataset with proxy labels
data_path = (
    "data/processed/enriched_transactions_with_proxy_labels.csv"  # Path to input data
)

# Initialize the loader class with the file path
loader = CreditDataLoader(filepath=data_path)  # Create instance of loader

# Load the dataset using robust error handling
try:
    df = loader.load()  # Load data into pandas DataFrame
    print(
        f"✅ Data loaded successfully: {df.shape[0]:,} rows × {df.shape[1]:,} columns"
    )  # Print row/column counts
except Exception as e:
    print(
        f"❌ Failed to load enriched transaction data: {e}"
    )  # Print error message if loading fails

✅ Transaction dataset loaded: 95,662 rows × 27 columns
✅ Data loaded successfully: 95,662 rows × 27 columns


## ⚙️ Apply Feature Engineering Pipeline (Task 4 – Feature Engineering)

This step applies the **modular, production-grade feature engineering pipeline** to the loaded enriched transaction dataset using the `DataProcessor` utility class.  
The pipeline transforms the raw proxy-labeled data into a **model-ready feature set** for downstream credit risk modeling and explainability.

The pipeline performs the following transformations:

- 📊 **Aggregate Feature Engineering:**  
  Customer-level behavioral features including transaction counts, totals, averages, variability (Amount, Value, RFM).
  
- 🕒 **Temporal Feature Extraction:**  
  Transaction patterns based on **hour of day**, **day of the week**, **month**, and **year**.

- 🔤 **Categorical Encoding:**  
  Encodes key categorical predictors using **Label Encoding** (default) or **One-Hot Encoding** (optional).

- 🩹 **Missing Value Handling & Scaling:**  
  Applies robust **median imputation** and **standardization** to numeric features using a reproducible **sklearn Pipeline**.

- 🧹 **Feature Set Assembly:**  
  Drops redundant identifiers and assembles a **clean, deduplicated dataset** ready for immediate model training and evaluation.

---

✅ This ensures compliance with the challenge brief’s requirements for **automated, reproducible, and explainable** feature engineering.


In [9]:
# ------------------------------------------------------------------------------
# ⚙️ Apply Feature Engineering Pipeline (Task 4 Processor with Verbose Diagnostics)
# ------------------------------------------------------------------------------

# Import the custom DataProcessor class from your src module
from src.labeling.data_processing import (
    DataProcessor,
)  # Provides full feature engineering pipeline

# Step 1️⃣ Initialize the DataProcessor with the loaded DataFrame
print("🔄 Initializing feature engineering processor...")

# Optional: Set use_one_hot=True if you need One-Hot Encoding instead of Label Encoding
processor = DataProcessor(
    df, use_one_hot=False
)  # Initialize DataProcessor with enriched transaction data

# Step 2️⃣ Run the full feature engineering pipeline with verbose diagnostics
try:
    print("🚀 Starting the feature engineering pipeline...")

    # Execute all feature engineering steps: aggregates, temporal, encoding, scaling, assembly
    final_df = processor.run_full_pipeline()

    print("✅ Feature engineering completed successfully.")
    print(
        f"🔢 Final dataset shape: {final_df.shape[0]:,} rows × {final_df.shape[1]:,} columns"
    )

    # Optional: Display first few rows for sanity check
    display(final_df.head())

except Exception as e:
    # Catch and display any pipeline errors
    print(f"❌ Feature engineering pipeline failed: {e}")

# ------------------------------------------------------------------------------
# ✅ Pipeline Overview (What this cell does):
# ------------------------------------------------------------------------------
# 1. 📊 Aggregate Feature Engineering → Customer-level behavior (transaction counts, averages, variance)
# 2. 🕒 Temporal Feature Extraction → Patterns by hour, day, month, year
# 3. 🔤 Categorical Encoding → Label or One-Hot encoding of key predictors
# 4. 🩹 Missing Value Handling & Scaling → Median imputation + Standardization
# 5. 🧹 Feature Set Assembly → Clean, deduplicated, model-ready dataset

🔄 Initializing feature engineering processor...
🚀 Starting the feature engineering pipeline...
🚀 Starting feature engineering pipeline...
✅ Aggregate features created.
✅ Temporal features extracted.
✅ Categorical variables encoded using Label Encoding.
✅ Missing values handled and numerical features scaled.
✅ Final dataset assembled. Shape: 77,761 rows × 39 columns.
✅ Feature engineering completed successfully.
🔢 Final dataset shape: 77,761 rows × 39 columns


,CurrencyCode,CountryCode,ProductCategory,Amount,Value,PricingStrategy,FraudResult,IsSharedAccount,Recency,Frequency,Monetary,FraudRisk,BehaviorCluster,BehavioralVarianceFlag,RiskScore,Amount_count,Amount_sum,Amount_mean,Amount_std,Amount_max,Amount_min,Value_sum,Value_mean,Value_std,FraudResult_sum,Recency_mean,Frequency_mean,Monetary_mean,AccountId,CustomerId,ProviderId,ProductId,ChannelId,risk_category,is_high_risk,TransactionHour,TransactionDayOfWeek,TransactionMonth,TransactionYear
0,0.00,0.00,-0.80,-0.05,-0.07,-0.35,-0.04,-0.82,-0.73,-0.31,-0.24,-0.04,-0.55,0.00,-0.50,-0.31,0.17,-0.07,-0.17,-0.19,0.16,-0.24,-0.10,-0.17,-0.10,-0.73,-0.31,-0.24,AccountId_3957,CustomerId_4406,5,ProductId_10,2,0,0,2,3,11,2018
1,0.00,0.00,0.49,-0.05,-0.08,-0.35,-0.04,1.23,-0.73,-0.31,-0.24,-0.04,-0.55,0.00,-0.50,-0.31,0.17,-0.07,-0.17,-0.19,0.16,-0.24,-0.10,-0.17,-0.10,-0.73,-0.31,-0.24,AccountId_4841,CustomerId_4406,3,ProductId_6,1,0,0,2,3,11,2018
2,0.00,0.00,-0.80,-0.05,-0.08,-0.35,-0.04,-0.82,3.34,-0.44,-0.25,-0.04,1.96,0.00,2.65,-0.44,0.17,-0.07,-0.20,-0.23,0.28,-0.25,-0.11,-0.20,-0.10,3.34,-0.44,-0.25,AccountId_4229,CustomerId_4683,5,ProductId_1,2,2,1,2,3,11,2018
3,0.00,0.00,4.36,0.11,0.10,-0.35,-0.04,-0.82,-0.52,-0.40,-0.24,-0.04,-0.55,0.00,-0.50,-0.40,0.18,-0.01,-0.01,-0.04,0.05,-0.24,-0.03,0.00,-0.10,-0.52,-0.40,-0.24,AccountId_648,CustomerId_988,0,ProductId_21,2,0,0,3,3,11,2018
4,0.00,0.00,0.49,-0.06,-0.08,-0.35,-0.04,1.23,-0.52,-0.40,-0.24,-0.04,-0.55,0.00,-0.50,-0.40,0.18,-0.01,-0.01,-0.04,0.05,-0.24,-0.03,0.00,-0.10,-0.52,-0.40,-0.24,AccountId_4841,CustomerId_988,3,ProductId_6,1,0,0,3,3,11,2018


In [10]:
# ------------------------------------------------------------------------------
# 💾 Save Final Model-Ready Data (Task 4 Output)
# ------------------------------------------------------------------------------

output_path = "data/processed/final_model_ready_data.csv"  # Define save path

try:
    final_df.to_csv(output_path, index=False)  # Save DataFrame to CSV
    print(
        f"✅ Model-ready dataset saved: {output_path} ({final_df.shape[0]:,} rows × {final_df.shape[1]:,} columns)"
    )
except Exception as e:
    print(f"❌ Failed to save model-ready dataset: {e}")

✅ Model-ready dataset saved: data/processed/final_model_ready_data.csv (77,761 rows × 39 columns)
